In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
import random

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import text, sequence
from keras.layers import Embedding, SpatialDropout1D
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.optimizers import RMSprop
import keras.backend as K
from keras.layers import Dense, Input, GRU, LSTM, Bidirectional, Dropout, CuDNNLSTM, CuDNNGRU, GlobalAveragePooling1D, GlobalMaxPool1D
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os

In [147]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#2. Get the file
data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

In [0]:
train['mal']         = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

X_unlab = np.array(test_labelled.comment_text)

In [0]:
#X = train.comment_text
#y = train.mal

max_features = 40000
maxlen       = 400
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
def gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units, reduction = 'average'):
    if K.backend == 'tensorflow':        
        K.clear_session()
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(max_features, output_dim=embed_dim, trainable=True)(input_layer)
    x = SpatialDropout1D(dropout_rate)(embedding_layer)
    x = Bidirectional(CuDNNGRU(units=rec_units, return_sequences=True))(x)
    if reduction == 'average':
      x = GlobalAveragePooling1D()(x)
    elif reduction == 'maximum':
      x = GlobalMaxPool1D()(x)
    elif reduction == 'attention':
      x = AttentionWithContext()(x)
      
    output_layer = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    #print( model.summary())
    return model

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)

### Average Pooling

### PSEUDO-LABELLING

In [0]:
not_trained=False
if not_trained:
  ps_labels = pd.DataFrame({'text'   : X_unlab, 
                            'model_1': np.zeros(len(X_unlab)), 
                            'model_2': np.zeros(len(X_unlab)), 
                            'model_3': np.zeros(len(X_unlab)),
                            'model_4': np.zeros(len(X_unlab)), 
                            'model_5': np.zeros(len(X_unlab))})

  for c, (train_index, val_index) in enumerate(kf.split(X, y)):
      print(f' fold {c}')

      X_train      = X[train_index]
      y_train      = y[train_index]

     
      if c==0:
         list_tokenized_train = tokenizer.texts_to_sequences(X_train)
         list_tokenized_unlab = tokenizer.texts_to_sequences(X_unlab)

      X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
      X_unlab              = sequence.pad_sequences(list_tokenized_unlab, maxlen=maxlen)

      model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)
      history              = model.fit(X_train, y_train, batch_size=batch_size, epochs=4, shuffle=False, verbose=1)
      probs_unlab          = model.predict(X_unlab, batch_size=batch_size, verbose=1)

      ps_labels.loc[:, [f'model_{c+1}']] = probs_unlab
      
      
  ps_labels['mean_score'] = ps_labels.drop('text', axis=1).mean(axis=1)
  ps_labels['mean_score_without_5fold'] = ps_labels.drop(['text', 'mean_score', 'model_5'], axis=1).mean(axis=1)
  ps_labels.to_csv(f'{data_path}/pseudo_labels_scores.csv', index=False)

In [0]:
ps_labels = pd.read_csv(f'{data_path}/pseudo_labels_scores.csv')
ps_labels['mean_score_without_5fold'] = ps_labels.drop(['text', 'mean_score', 'model_5'], axis=1).mean(axis=1)
ps_labels['y'] = 1
ps_labels.loc[ps_labels['mean_score_without_5fold']< 0.1, 'y'] = 0

In [0]:
X = pd.concat([pd.Series(X), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.6, 'text']], axis=0)
y = pd.concat([pd.Series(y), ps_labels.loc[ps_labels['mean_score_without_5fold']>0.6, 'y']], axis=0)

#X = pd.concat([pd.Series(X), ps_labels.loc[ps_labels['mean_score_without_5fold']< 0.1 , 'text']], axis=0)
#y = pd.concat([pd.Series(y), ps_labels.loc[ps_labels['mean_score_without_5fold']< 0.1, 'y']], axis=0)

In [0]:
X = np.array(X)
y = np.array(y)

In [157]:
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    #tokenizer = text.Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(pd.concat([X_train, unlab.comment_text], axis=0))
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    list_tokenized_val   = tokenizer.texts_to_sequences(X_val)
    X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_val                = sequence.pad_sequences(list_tokenized_val, maxlen=maxlen)
    model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=False, verbose=1)
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    model.save_weights(f'{cv_models_path}/BGRU_avpool_SEMI_fold_06_{c}.h5')
    
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    del model

 fold 0
Fitting
Train on 129893 samples, validate on 32474 samples
Epoch 1/4
129893/129893 [==============================] - 63s 486us/step - loss: 0.2755 - acc: 0.9091 - val_loss: 0.9516 - val_acc: 0.4759
Epoch 2/4
129893/129893 [==============================] - 61s 469us/step - loss: 0.1285 - acc: 0.9527 - val_loss: 0.2161 - val_acc: 0.9272
Epoch 3/4
129893/129893 [==============================] - 61s 471us/step - loss: 0.1090 - acc: 0.9593 - val_loss: 0.1358 - val_acc: 0.9574
Epoch 4/4
32474/32474 [==============================] - 5s 168us/step
 average precision 0.8936407980761089
 roc auc 0.970996517697224
 fold 1
Fitting
Train on 129893 samples, validate on 32474 samples
Epoch 1/4
129893/129893 [==============================] - 63s 486us/step - loss: 0.2596 - acc: 0.9123 - val_loss: 3.1100 - val_acc: 0.1436
Epoch 2/4
129893/129893 [==============================] - 61s 468us/step - loss: 0.2339 - acc: 0.9246 - val_loss: 0.6933 - val_acc: 0.6987
Epoch 3/4
129893/129893 [=====

In [0]:
data = pd.DataFrame({'acc':history.history['acc'],
                    'loss': history.history['loss'],
                    'val_acc': history.history['val_acc'],
                    'val_loss': history.history['val_loss']})

In [159]:
np.array(auc).mean()

0.8968111613872789

In [160]:
np.array(roc).mean()

0.976435779690917

In [0]:
X_train   = X
y_train   = y
tokenizer = text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test  = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [162]:
model   = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units, reduction='average')

y_train = np.array(y_train)
y_test  = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train,   batch_size=batch_size, epochs=4, shuffle=True, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
model.save_weights(f'{models_path}/BGRU_SEMI_06.h5')
del model

Fitting
Epoch 1/4
162367/162367 [==============================] - 72s 446us/step - loss: 0.2138 - acc: 0.9249
Epoch 2/4
162367/162367 [==============================] - 70s 431us/step - loss: 0.1179 - acc: 0.9556
Epoch 3/4
162367/162367 [==============================] - 70s 431us/step - loss: 0.1095 - acc: 0.9579
Epoch 4/4
64830/64830 [==============================] - 11s 162us/step


In [163]:
auc_f

0.8573728620502495

In [164]:
roc_f

0.9733895638613443